In [1]:
import numpy as np
from scipy.special import expit
from evaluator_Linear import evaluator
from probLearner import PMLearner, RewardLearner, PALearner
from ratioLearner import  RatioLinearLearner as RatioLearner
from qLearner_Linear import Qlearner

## STEP0： Define the Target Policy and the Control Policy

Modify the Policy.py file accordingly

In [2]:
from Policy import target_policy, control_policy

## STEP1: prepare the dataset

The following is an example of a proper input dataset with 2 trajectories and 3 observations for each trajectory, which is a dictionary with keys:
- s0: stacked initial states of all the trajectories, initial state, 2d-array
- state: stacked states of all the trajectories at all time points, 2d-array
- action: stacked sequence of actions for all trajectories at all time points, 1d-array
- mediator: stacked mediators of all the trajectories at all time points, 2d-array
- reward: stacked sequence of rewards for all trajectories at all time points, 1d-array
- next_state: stacked next_states of all the trajectories at all time points, 2d-array

In [11]:
#from Simulator import Simulator
#dim_state=3; dim_mediator = 2
#simulator = Simulator(model_type='Gaussian_semi', dim_state=dim_state, dim_mediator = dim_mediator)
#simulator.sample_trajectory(num_trajectory=2, num_time=3, seed=0)
#simulator.trajectory2iid()
#sim_iid_dataset = simulator.iid_dataset
#dataset = sim_iid_dataset
#dataset

{'s0': array([[ 1.76405235,  0.40015721,  0.97873798],
        [-0.18158257,  1.41020463, -0.37447169]]),
 'state': array([[ 1.76405235,  0.40015721,  0.97873798],
        [ 2.42993277, -0.32779064, 12.83547844],
        [ 1.4532285 , -1.75914681, 23.4816176 ],
        [-0.18158257,  1.41020463, -0.37447169],
        [ 1.47498535, -1.33984802,  4.4633054 ],
        [-0.04107094,  0.74313238,  7.3301179 ]]),
 'action': array([0., 1., 1., 1., 0., 1.]),
 'mediator': array([[ 2.42993277, -0.32779064],
        [ 1.4532285 , -1.75914681],
        [ 1.89892964, -3.8330894 ],
        [ 1.47498535, -1.33984802],
        [-0.04107094,  0.74313238],
        [ 1.14614205,  0.18356083]]),
 'reward': array([12.83547844, 23.4816176 , 32.8543457 ,  4.4633054 ,  7.3301179 ,
        19.61499902]),
 'next_state': array([[ 2.42993277, -0.32779064, 12.83547844],
        [ 1.4532285 , -1.75914681, 23.4816176 ],
        [ 1.89892964, -3.8330894 , 32.8543457 ],
        [ 1.47498535, -1.33984802,  4.4633054 ],

## STEP2: Modify the hyper-parameters

In [8]:
#Fixed hyper-parameter--no need to modify
expectation_MCMC_iter = 100
expectation_MCMC_iter_Q3 = expectation_MCMC_iter_Q_diff = 50
truncate = 100
problearner_parameters = {"splitter":["best","random"], "max_depth" : range(1,40)},

#hyperparameters that need modification
#dim_state = the dimension of the state variable
#dim_meditor = the dimension of the mediator variable
#ratio_ndim = number of features used to learn the ratio model # can be modified accordingly 
                #to learn how the ratio_ndim affect the estimation performance
dim_state=3; dim_mediator = 2
ratio_ndim = 15

## STEP3: Causal Effect Estimation

In [ ]:
est_obj1 = evaluator(dataset, Qlearner, PMLearner, RewardLearner, PALearner, RatioLearner,
                     target_policy=target_policy, control_policy = control_policy, dim_state = dim_state,
                     dim_mediator = dim_mediator, scaler = 'Identity', problearner_parameters = problearner_parameters,
                     expectation_MCMC_iter = expectation_MCMC_iter, expectation_MCMC_iter_Q3 = expectation_MCMC_iter_Q3, 
                     expectation_MCMC_iter_Q_diff = expectation_MCMC_iter_Q_diff,
                     ratio_ndim = ratio_ndim, truncate = truncate, 
                     seed = 10)

est_obj1.estimate_DE_ME_SE()
est_value1 = est_obj1.est_DEMESE
var_value1 = est_obj1.var_DEMESE


#The following are the estimations of our interest

#1. estimation used the proposed triply robust estimator
DE_TR, ME_TR, SE_TR = est_value1[:3]

#2. estimation used the direct estimator of etas
DE_Direct, ME_Direct, SE_Direct = est_value1[3:6]

#3. estimation used the baseline method
DE_base, ME_base = est_value1[6:8]
SE_base= np.nan